In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

### Read data from csv

In [ ]:
# Load your time series data
data = pd.read_csv('13 Week Treasury Bill.csv', usecols=[0, 1, 2])

# Reformat Dates as indices
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

### Take average prices

In [ ]:
# Take average of High and Low prices
data['Average'] = (data['High'] + data['Low']) / 2

# Separate training and test data
y_train = data[:5000]
y_test = data[5000:]

print(data)

In [ ]:
# Plot average of data
plt.figure(figsize=(12,8))
plt.plot(y_train['Average'], label='Training Data', color='blue')
plt.plot(y_test['Average'], label='Test Data', color='red')
plt.title("13 Week Treasury Bill Yield Rates")
plt.ylabel("Rates as Percentages")
plt.xlabel("Year")
plt.legend()
plt.show()

### Run statistical tests for data preprocessing

In [ ]:
ADF_result = adfuller(y_train['Average'])
print('ADF Statistic: %f' % ADF_result[0])
print('p-value: %f' % ADF_result[1])

Data is non-stationary, indicated by a high p-value

In [ ]:
# Apply first-order differencing to remove non-stationarity
first_diff = y_train['Average'].diff()[1:]
plt.figure(figsize=(12,6))
plt.plot(first_diff, label='First-Order Differenced Data', color='blue')
plt.show()

### Run ACF and PACF for lag correlation

In [ ]:
acf_plot = plot_acf(first_diff, lags=25)

In [ ]:
pacf_plot = plot_pacf(first_diff)

### Fit ARIMA(p,q) model

In [ ]:
my_order = (1,0,2)

In [ ]:
# Fit the ARIMA model
model = ARIMA(y_train['Average'], order=my_order)
results = model.fit()
print(results.summary())

### Use model to predict on test time series

In [ ]:
# Forecast predictions from ARIMA model
predictions = results.forecast(len(y_test))
predictions.index = y_test.index
predictions = pd.Series(predictions, index=y_test.index)
residuals = y_test['Average'] - predictions

In [ ]:
# Plot residuals
plt.figure(figsize=(12,6))
plt.plot(residuals)
plt.axhline(0, linestyle='--', color='k')
plt.title('Residuals from SARIMA Model', fontsize=20)
plt.ylabel('Error', fontsize=16)
plt.show()

In [ ]:
# Plot predictions vs Actual Data
plt.figure(figsize=(12,6))
plt.plot(data['Average'], label='Data', color='blue')
plt.plot(predictions, label='Predictions', color='red')
plt.legend()
plt.show()

### Visualize predictions

In [ ]:
print(predictions)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(predictions, label='Predictions', color='red')
plt.legend()
plt.show()